**Goal:** represent the tracks downloaded from [Wikiloc](http://wikiloc.com/) or transformed into .gpx format from Garmin .fit files, using for example the [gpsbabel](http://www.gpsbabel.org/) tool.

In [2]:
import re
import os
import folium
import glob
import logging
import numpy as np
from geopy.geocoders import Nominatim
from folium import plugins
import matplotlib.pyplot as plt
from matplotlib import colors

/usr/local/lib/python2.7/dist-packages/matplotlib/__init__.py:872: UserWarning: axes.color_cycle is deprecated and replaced with axes.prop_cycle; please use the latter.
  warnings.warn(self.msg_depr % (key, alt_key))


In [3]:
print folium.__version__

0.2.1


# User inputs

Define the directory where the files are stored and where the html will be written.

In [11]:
gpxdir = '/home/ctroupin/DataOceano/GPX/Comparison/'
outputdir = '/home/ctroupin/public_html/LeafletMaps'
mapname = '20160804_comparison.html'
gpxbasename = '*.gpx'
gpxfilelist = sorted(glob.glob(os.path.join(gpxdir, gpxbasename)))
colorlist = ['red', 'blue', 'green']
namelist = ['Suunto', 'Garmin', 'Wikiloc']

Initialise the map

In [5]:
import logging
logger = logging.getLogger()
fhandler = logging.FileHandler(filename='/home/ctroupin/logs/GPXheadmap.log', mode='a')
formatter = logging.Formatter('%(asctime)s - %(name)s - %(levelname)s - %(message)s')
fhandler.setFormatter(formatter)
logger.addHandler(fhandler)
logger.setLevel(logging.DEBUG)

geolocator = Nominatim()
location = geolocator.geocode("Parc Bit, Mallorca, Spain")
logger.info("Working on location: %s" %(location[0]))

Function to read GPX

In [6]:
def read_gpx(filename):
    coords = []
    with open(filename) as f:
        for lines in f:
            match = re.search('<trkpt lat="([-0-9\.]+)" lon="([-0-9\.]+)">', lines)
            if match:
                coords.append((float(match.group(1)), float(match.group(2))))
    return coords

Loop on the files

In [13]:
logger.info('Creating a new map')
map_run = folium.Map(location=[location.latitude, location.longitude], 
                     zoom_start=12)

logger.info('Loop on the GPX files')
for gpxfiles, color, name in zip(gpxfilelist, colorlist, namelist):
    logger.debug(gpxfiles)
    coords = read_gpx(gpxfiles)
    folium.Marker(coords[0], popup=name).add_to(map_run)
    folium.PolyLine(coords, color=color, weight=7).add_to(map_run)
    
map_run

In [10]:
if not(os.path.exists(outputdir)):
    os.makedirs(outputdir)
    logging.info('Creating directory %s' %(outputdir))
map_run.save(os.path.join(outputdir, mapname))